In [1]:
import sqlite3

import pandas as pd

db_conn = sqlite3.connect('/home/orestis/code/Orestis/InvestorAPI/app/database/ibd.db')

In [25]:
query = "SELECT * FROM stocks_with_sector"
df = pd.read_sql(query, db_conn)

In [11]:
df

,comp_rating,eps_rating,rs_rating,acc_dis_rating,fifty_two_wk_high,name,symbol,closing_price,price_change_pct,vol_chg_pct,smr_rating,sector_name,sector_daily_price_change_pct,sector_start_of_year_price_change_pct,registered_date,registered_date_ts
0,97,99,97,B,22.1,ComstckRes,CRK,18.90,-0.34,-76.0,A,ENERGY,0.4,49.23,25-11-2022,1669327200
1,97,99,95,B,73.8,Matador,MTDR,66.54,-0.20,-82.0,A,ENERGY,0.4,49.23,25-11-2022,1669327200
2,95,81,97,B,36.5,Brigham Minrls,MNRL,35.29,0.16,-66.0,A,ENERGY,0.4,49.23,25-11-2022,1669327200
3,91,80,93,B-,64.2,Cactus A,WHD,52.89,-0.79,-62.0,A,ENERGY,0.4,49.23,25-11-2022,1669327200
4,97,91,98,A-,32.4,ShoalsTech,SHLS,31.03,0.15,-73.0,..,ENERGY,0.4,49.23,25-11-2022,1669327200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175501,66,88,59,B,70.9,MolsonCoorsB,TAP,61.80,-1.25,206.0,C,ALCOHL/TOB,-1.2,-24.38,15-12-2023,1702591200
175502,42,79,32,C-,105.0,PhillipMrris,PM,94.46,-0.21,270.0,..,ALCOHL/TOB,-1.2,-24.38,15-12-2023,1702591200
175503,70,86,71,C+,26.3,Turning Point,TPB,24.45,-0.37,71.0,B,ALCOHL/TOB,-1.2,-24.38,15-12-2023,1702591200
175504,75,54,89,A+,NaN,Universal,UVV,63.32,-0.10,999.0,C,ALCOHL/TOB,-1.2,-24.38,15-12-2023,1702591200


In [12]:
df.columns

Index(['comp_rating', 'eps_rating', 'rs_rating', 'acc_dis_rating',
       'fifty_two_wk_high', 'name', 'symbol', 'closing_price',
       'price_change_pct', 'vol_chg_pct', 'smr_rating', 'sector_name',
       'sector_daily_price_change_pct',
       'sector_start_of_year_price_change_pct', 'registered_date',
       'registered_date_ts'],
      dtype='object')

In [26]:
# Convert 'registered_date' to datetime
df['registered_date'] = pd.to_datetime(df['registered_date'], format='%d-%m-%Y')

# Sort the DataFrame based on 'symbol' and 'registered_date'
df.sort_values(by=['symbol', 'registered_date'], inplace=True)

df['closing_price_after_1_w'] = df.groupby('symbol')['closing_price'].shift(-1)
# Calculate the percentage change
df['price_pct_change_after_1_w'] = ((df['closing_price_after_1_w'] - df['closing_price']) / df['closing_price']) * 100

df['eps_rating_change'] = df.groupby('symbol')['eps_rating'].diff()
df['comp_rating_change'] = df.groupby('symbol')['comp_rating'].diff()
df['rs_rating_change'] = df.groupby('symbol')['rs_rating'].diff()
df['fifty_two_wk_high_pct_change'] = df.groupby('symbol')['fifty_two_wk_high'].pct_change() * 100

# Create categorical target column
bins = [-float('inf'), -10, -5, 0, 5, 10, float('inf')]
labels = ['-10-', '-5 to -10', '-5 to 0', '0 to 5', '5 to 10', '10+']

df['price_pct_change_after_1_w_range'] = pd.cut(
    df['price_pct_change_after_1_w'],
    bins=bins,
    labels=labels,
    right=False
)

df.drop([
    'comp_rating',
    'eps_rating',
    'rs_rating',
    'name',
    'symbol',
    'fifty_two_wk_high',
    'price_change_pct',
    'registered_date_ts',
    'closing_price_after_1_w',
    'closing_price',
    'price_pct_change_after_1_w'
], axis=1, inplace=True)

df = df.dropna()
# Reset index if needed
df.reset_index(drop=True, inplace=True)


In [27]:
df

,acc_dis_rating,closing_price,vol_chg_pct,smr_rating,sector_name,sector_daily_price_change_pct,sector_start_of_year_price_change_pct,registered_date,eps_rating_change,comp_rating_change,rs_rating_change,fifty_two_wk_high_pct_change,price_pct_change_after_1_w_range
0,A,133.60,-57.0,A,MEDICAL,1.0,-15.48,2022-08-05,0.0,-1.0,-6.0,0.0,0 to 5
1,A,133.60,-40.0,A,MEDICAL,1.4,-13.36,2022-08-12,0.0,-3.0,-12.0,0.0,0 to 5
2,A-,137.60,-3.0,A,MEDICAL,-0.3,-14.95,2022-08-19,2.0,6.0,2.0,0.0,-5 to -10
3,A-,129.90,-22.0,A,MEDICAL,-3.1,-17.64,2022-08-26,0.0,-7.0,-7.0,0.0,-5 to 0
4,B+,128.00,18.0,A,MEDICAL,-1.2,-18.95,2022-09-02,0.0,-2.0,4.0,0.0,5 to 10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
167271,C-,12.00,55.0,B,MEDICAL,1.2,-4.45,2023-03-31,0.0,2.0,1.0,0.0,0 to 5
167272,B,12.16,-26.0,B,MEDICAL,0.4,-3.57,2023-04-06,0.0,5.0,2.0,0.0,-5 to 0
167273,B+,11.94,-62.0,B,MEDICAL,-0.5,-1.94,2023-04-14,0.0,-1.0,-4.0,0.0,0 to 5
167274,B+,11.96,-49.0,B,MEDICAL,1.5,-0.58,2023-04-21,0.0,1.0,2.0,0.0,-5 to 0
